In [1]:
# State

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_backtracking import ProjectedGradientDescentBacktracking, ProjectedGradientDescentBacktrackingOption
from quara.data_analysis.weighted_probability_based_squared_error import WeightedProbabilityBasedSquaredError, WeightedProbabilityBasedSquaredErrorOption
from quara.data_analysis.weighted_relative_entropy import WeightedRelativeEntropy, WeightedRelativeEntropyOption
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_povm,
    get_y_povm,
    get_z_povm,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import LossMinimizationEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

from quara.data_analysis.simulation import StandardQTomographySimulationSetting
from quara.settings import Settings

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
start_all = time.time()

In [4]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_povm(c_sys)
povm_y = get_y_povm(c_sys)
povm_z = get_z_povm(c_sys)
tester_objects = [povm_x, povm_y, povm_z]

In [5]:
# Case 1:
true_object = get_z0_1q(c_sys)

# Case 2:
# vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
# true_object = State(c_sys, vec)

# Case 3:
# vec = np.array([1 / np.sqrt(2), 0, 0, 0], dtype=np.float64)
# true_object = State(c_sys, vec)

true_object.vec

array([0.70710678, 0.        , 0.        , 0.70710678])

In [6]:
num_data = [100, 1000, 10000]
# n_rep = 100
n_rep = 10

case_name_list = [
    "Linear(True)",
    "Linear(False)",
    "ProjectedLinear(True)",
    "ProjectedLinear(False)",
    "Maximum-Likelihood(True)",
    "Maximum-Likelihood(False)",
    "Least Squares(True)",
    "Least Squares(False)",
]

seed = 777
qtomography_list = [
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
]
para_list = [
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
]

loss_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropy(3),
    WeightedRelativeEntropy(4),
    WeightedProbabilityBasedSquaredError(3),
    WeightedProbabilityBasedSquaredError(4),
]

loss_option_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropyOption("identity"),
    WeightedRelativeEntropyOption("identity"),
    WeightedProbabilityBasedSquaredErrorOption("identity"),
    WeightedProbabilityBasedSquaredErrorOption("identity"),
]

algo_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
]

algo_option_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
        mode_proj_order=order
    ),
    ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
        mode_proj_order=order
    ),
    ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
        mode_proj_order=order
    ),
    ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
    mode_proj_order=order
    ),
]

estimation_results_list = []
elapsed_times = []
simulation_settings = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    loss = loss_list[i]
    loss_option = loss_option_list[i]
    algo = algo_list[i]
    algo_option = algo_option_list[i]

    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
        qtomography=qtomography,
        true_object=true_object,
        num_data=num_data,
        estimator=estimator,
        loss=loss,
        loss_option=loss_option,
        algo=algo,
        algo_option=algo_option,
        iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)

    # stock settings of this simulation
    simulation_setting = StandardQTomographySimulationSetting(
        name=name,
        estimator=estimator,
        loss=loss,
        loss_option=loss_option,
        algo=algo,
        algo_option=algo_option,
        true_object=true_object,
        tester_objects=tester_objects
    )
    simulation_settings.append(simulation_setting)

    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

 10%|█         | 1/10 [00:00<00:00,  9.61it/s]

Case 0: Linear(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: LinearEstimator


 20%|██        | 2/10 [00:00<00:00, 10.52it/s]

elapsed_time:0.01579508384068807[min]

Case 1: Linear(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: LinearEstimator


 10%|█         | 1/10 [00:00<00:01,  8.37it/s]

...

elapsed_time:0.14997086922327676[min]

Case 6: Least Squares(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: LossMinimizationEstimator


  0%|          | 0/10 [00:00<?, ?it/s]

elapsed_time:0.03425743182500204[min]

Case 7: Least Squares(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: LossMinimizationEstimator


100%|██████████| 10/10 [00:02<00:00,  4.58it/s]

elapsed_time:0.03644274473190308[min]



In [7]:
report.export_report(f"qst_nrep={n_rep}.pdf",
    estimation_results_list=estimation_results_list,   # 「EstimationResultのリスト」のリスト
    simulation_settings= simulation_settings,
    seed=seed,  # 推定で使ったseed（オプション）
    tolerance=Settings.get_atol()
)

​Generating table of computation time ...
​Generating table of tolerance of physicality constraint violation ...
​Generating table of experimental conditions ...
Generating case list ...
Computation time of estimators ...


100%|██████████| 10/10 [00:00<00:00, 21355.93it/s]

​​Generating MSE of empirical distributions blocks ...
​​Generating consictency test blocks ...


​Generating a graph for MSE ...
  Linear(True)=[0.012880000000000011, 0.0011730000000000017, 0.00013258400000000013]
  Linear(False)=[0.009720000000000008, 0.0013102000000000025, 4.586600000000015e-05]
  ProjectedLinear(True)=[0.015939715201437006, 0.0006260694002287519, 6.762450519438463e-05]
  ProjectedLinear(False)=[0.008147752343453163, 0.0009049210586556524, 0.00017207082262236083]
  Maximum-Likelihood(True)=[0.005997859417841656, 0.0006176245267523696, 4.109059699343727e-05]
  Maximum-Likelihood(False)=[0.005056869585468227, 0.00030141774741704444, 3.950518415951611e-05]
  Least Squares(True)=[0.011737443300339477, 0.0007407001244195064, 9.298807724194137e-05]
  Least Squares(False)=[0.00565886127025111, 0.0015894545151952538, 5.127381669327336e-05]
  Linear(True)=[0.012880000000000011, 0.0011730000000000017, 0.00013258400000000013]
  ProjectedLinear(True)=[0.015939715201437006, 0.0006260694002287519, 6.762450519438463e-05]
  Maximum-Likelihood(True)=[0.005997859417841656, 0.0006

100%|██████████| 10/10 [00:00<00:00, 4376.36it/s]


Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (qst_nrep=10.pdf)


In [8]:
report.export_report?